In [1]:
from pathlib import Path
from collections import defaultdict
import mat_ceng as mc
import logging
import contextlib
import pandas as pd

In [2]:
folder_name = Path('.').resolve()
file_name = folder_name / 'sample_warning.txt'

result = {
    'total_warnings': 0,
    'colleced_warnings': 0,
    'warnings': defaultdict(list),  # Using defaultdict for automatic list initialization,
    'warning_locations': []
}

unit_conversion = 1000.0

# Read file line by line
with open(file_name, 'r') as file:
    for indx, line in enumerate(file):
        if indx == 0:
            result['total_warnings'] = int(line.strip().split(', ')[1].split(' ')[0])
            continue
        row = line.strip()
        if 'Check at (' in row:
            result['colleced_warnings'] += 1
            coord = row.split('Check at (')[1].split(')')[0].split(' ')
            coord = [float(x) * unit_conversion for x in coord]
            coord = tuple(coord)
            result['warning_locations'].append(coord)
            result['warnings'][coord].append(row)

result['warnings'] = dict(result['warnings'])  # Convert defaultdict to dict
result['warning_locations'] = list(set(result['warning_locations']))  # Remove duplicates
print(f'total warnings: {result["total_warnings"]}, collected warnings: {result["colleced_warnings"]}')


total warnings: 323, collected warnings: 323


In [3]:
mc.csi.CsiHelper.initialize()
mc.csi.CsiHelper.connect_to_etabs(unit='N_mm_C')


2025-04-26 14:57:34 - mat_ceng.csi - INFO - Initializing CsiHelper...
2025-04-26 14:57:34 - mat_ceng.csi - INFO - SAP2000 library loaded successfully from: C:\Program Files\Computers and Structures\SAP2000 26\SAP2000v1.dll
2025-04-26 14:57:34 - mat_ceng.csi - INFO - ETABS library loaded successfully from: C:\Program Files\Computers and Structures\ETABS 22\ETABSv1.dll
2025-04-26 14:57:34 - mat_ceng.csi - INFO - CsiHelper initialization complete.
2025-04-26 14:57:34 - mat_ceng.csi - INFO - Attempting to attach to running ETABS instance...
2025-04-26 14:57:34 - mat_ceng.csi - INFO - Successfully attached to running ETABS instance.
2025-04-26 14:57:34 - mat_ceng.csi - INFO - Setting initial ETABS units to N_mm_C
2025-04-26 14:57:34 - mat_ceng.csi - INFO - ETABS SapModel obtained and connection established.


In [4]:
warning_group_name = 'Warnings'
mc.csi.create_etabs_group(warning_group_name)

2025-04-26 14:57:34 - mat_ceng.csi - INFO - Successfully retrieved 2 ETABS groups.


'Warnings'

In [5]:
#logging.getLogger().setLevel(logging.WARNING)

In [6]:
# Create a context manager to temporarily change log level
@contextlib.contextmanager
def suppress_logs(log_level=logging.WARNING):
    root_logger = logging.getLogger()
    original_level = root_logger.level
    root_logger.setLevel(log_level)
    try:
        yield
    finally:
        root_logger.setLevel(original_level)

# Use it with your code
with suppress_logs():
    etabs_added_areas = [mc.csi.add_etabs_warning_mark(x) for x in result['warning_locations']]
etabs_added_areas[:3]

['339', '340', '341']

In [7]:
df = pd.DataFrame(result['warnings'].items(), columns=['Coordinates', 'Warnings'])
df['area_name'] = etabs_added_areas
df['Warnings'] = df['Warnings'].apply(lambda x: '\n'.join(x))
df.set_index('area_name', inplace=True)
df.to_excel('warnings.xlsx', index=True)
df.head(5)

,Coordinates,Warnings
area_name,,
339,"(35870.0, -33723.0, 16500.0)","UPPER ROOF FLOOR PLAN, B519 & B1657 are overla..."
340,"(33700.0, -35846.0, 16500.0)","UPPER ROOF FLOOR PLAN, B521 & B1658 are overla..."
341,"(-3799.0, -43703.0, 16500.0)","UPPER ROOF FLOOR PLAN, B79 & B1665 are too clo..."
342,"(26744.0, -48040.0, 16500.0)","UPPER ROOF FLOOR PLAN, B208 & B1659 are too cl..."
343,"(-21356.0, -45798.0, 16500.0)","UPPER ROOF FLOOR PLAN, B427 & B1666 are too cl..."


In [8]:
mc.csi.CsiHelper.release_connection()

2025-04-26 14:57:34 - mat_ceng.csi - INFO - Releasing CSI connection...
2025-04-26 14:57:34 - mat_ceng.csi - INFO - CSI connection references have been released.
